In [ ]:
import sys
sys.path.append("..")

import random
import math
import time
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union, Dict

import PIL.Image
import PIL.ImageDraw

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.encoder import *
from src.models.decoder import *
from src.models.util import *
from experiments import datasets

# load dataset

In [ ]:
SHAPE=(1, 28, 28)
ds = datasets.mnist_dataset(train=False, shape=SHAPE)

# load model

In [ ]:
model = EncoderConv2d(SHAPE, code_size=128, channels=(16, 16, 16), kernel_size=7, dropout=0.)
#model.load_state_dict(torch.load("../checkpoints/aug/rot1_ks-7_ch-16,16,16_drop-0.0/snapshot.pt")["state_dict"])
print(f"{num_module_parameters(model):,}")

## dataset -> features

In [ ]:
labels = []
with torch.no_grad():
    features = []
    for batch in tqdm(DataLoader(ds, batch_size=128)):
        features.append(
            model(batch[0])
        )
        labels.append(batch[1])
    features = torch.concat(features)
    labels = torch.concat(labels)
features.shape, labels.shape

# fit labels

In [ ]:
from sklearn.svm import SVC, SVR

classifier = SVC()
classifier.fit(features, labels)

In [ ]:
classifier.score(features, labels)
#classifier.predict(features)

In [ ]:
classifier.score?

In [ ]:
[i for i in range(1, 29) if 28/i == 28//i] 

In [ ]:
from torchvision.models.vision_transformer import vit_h_14
model = vit_h_14()
print(f"{num_module_parameters(model):,}")
model